# Session 12

## Machine Learning in the Cloud

![Course Hero](images/hero.png)

## Machine Learning Services in the Cloud

![AWS ML Stack](images/aws-ml-stack.jpg)


In [ ]:
from pprint import pprint
import io

import numpy as np
import pandas as pd
from PIL import Image, ImageDraw, ImageFont

import boto3

We need and Access Key Id and Access Key Secret to authenticate with AWS, use yours or ask for it to your instructor.

## WARNING: You should NEVER store credentials in a repository. Never save the notebook with the credentials!


In [ ]:
access_id = ""
secret_key = ""


We declare a session (its not opened until we use it) using that access key and the us-west-2 (Oregon) AWS region.


In [ ]:
aws_session = boto3.Session(
    aws_access_key_id = access_id,
    aws_secret_access_key = secret_key,
    region_name = "us-west-2"
)


## Rekognition

![Rekognition](images/rekognition.png)

We open a client to the Rekognition service, Artificial Intelligence vision.


In [ ]:
rekognition_client = aws_session.client('rekognition')


We set up the image name


In [ ]:
image_name = "images/sample.jpg"


We load the an image.


In [ ]:
image_display = Image.open(image_name)
display(image_display)


Now we will identify objects in the image (AWS calls it "tagging"). We load the image and send it out to AWS for analysis.


In [ ]:
with open(image_name, 'rb') as image:
    response = rekognition_client.detect_labels(Image={'Bytes': image.read()})


The response is a dictionary.


In [ ]:
pprint(response, depth=2)


In [ ]:
pprint(response["Labels"], depth=2)


In [ ]:
pprint(response["Labels"][0])


Now the fun begins...


In [ ]:
# Prepares the font and drawing canvas
font = ImageFont.truetype("Sudo-Bold.ttf", size=20)
draw = ImageDraw.Draw(image_display)
w, h = image_display.size


In [ ]:
# Get one label information
name = response["Labels"][0]["Name"]
print("Label Name: ", name)
instance = response["Labels"][0]["Instances"][0]
print("Instance: ")
pprint(instance)
bbox = instance['BoundingBox']

# Transform the coordinates from proportional to absolute
x0 = int(bbox['Left'] * w)
y0 = int(bbox['Top'] * h)
x1 = x0 + int(bbox['Width'] * w)
y1 = y0 + int(bbox['Height'] * h)
print("Absolute bbox:", x0, y0, x1, x1)


In [ ]:
# We draw a rectangle and a label over the canvas
draw.rectangle(
    (x0, y0, x1, y1),
    outline=(255, 0, 0),
    width=2
)
draw.text(
    (x0, y0),
    name,
    font=font,
    fill=(0, 255, 255)
)
display(image_display)


In [ ]:
# Now let's do it for all labels
for label in response['Labels']:
    name = label['Name']
    for instance in label['Instances']:
        bbox = instance['BoundingBox']
        x0 = int(bbox['Left'] * w)
        y0 = int(bbox['Top'] * h)
        x1 = x0 + int(bbox['Width'] * w)
        y1 = y0 + int(bbox['Height'] * h)
        draw.rectangle((x0, y0, x1, y1), outline=(255, 0, 0), width=2)
        draw.text((x0, y0), name, font=font, fill=(0, 255, 255))

display(image_display)


## Polly

We can also synthesize audio.

![AWS Polly](images/polly.jpg)


In [ ]:
from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import subprocess
from tempfile import gettempdir

polly = aws_session.client("polly")

try:
    # Request speech synthesis
    response = polly.synthesize_speech(
        Text="Machine learning is a great topic",
        OutputFormat="mp3",
        VoiceId="Joanna",
    )
    # We can also try Emma, or in Spanish Mia, Lupe o Lucia
    # All the supported languages and voices are documented at https://docs.aws.amazon.com/polly/latest/dg/voicelist.html
except (BotoCoreError, ClientError) as error:
    # The service returned an error, exit gracefully
    print(error)


In [ ]:
# Access the audio stream from the response
output = ""
if "AudioStream" in response:
    # Note: Closing the stream is important because the service throttles on the
    # number of parallel connections. Here we are using contextlib.closing to
    # ensure the close method of the stream object will be called automatically
    # at the end of the with statement's scope.
    with closing(response["AudioStream"]) as stream:
        output = os.path.join(gettempdir(), "speech.mp3")

        try:
        # Open a file for writing the output as a binary stream
            with open(output, "wb") as file:
                file.write(stream.read())
        except IOError as error:
            # Could not write to file, exit gracefully
            print(error)
            sys.exit(-1)

else:
    # The response didn't contain audio data, exit gracefully
    print("Could not stream audio")

# Play the audio using the platform's default player
if sys.platform == "win32":
    # The following works on Windows
    os.startfile(output)
else:
    # The following works on macOS and Linux. (Darwin = mac, xdg-open = linux).
    opener = "open" if sys.platform == "darwin" else "xdg-open"
    subprocess.call([opener, output])


## Transcribe

And we can do the opposite, take an audio file an transcribe it into text.

![Transcribe Logo](images/transcribe.jpg)


In [ ]:
import time

transcribe_client = aws_session.client('transcribe')

transcription_job_name="Must-be-a-unique-name"
transcript_uri = ""

transcribe_client.start_transcription_job(
    TranscriptionJobName=transcription_job_name,
    Media={"MediaFileUri": "s3://david21-publica/sample.mp3"},
    MediaFormat="mp3",
    LanguageCode="en-US"
)

max_tries = 60
while max_tries > 0:
    max_tries -= 1
    job = transcribe_client.get_transcription_job(TranscriptionJobName=transcription_job_name)
    job_status = job['TranscriptionJob']['TranscriptionJobStatus']
    if job_status in ['COMPLETED', 'FAILED']:
        print(f"Job status is {job_status}.")
        if job_status == 'COMPLETED':
            transcript_uri = job['TranscriptionJob']['Transcript']['TranscriptFileUri']
            print(
                f"Download the transcript from\n"
                f"\t{transcript_uri}.")
        break
    else:
        print(f"Waiting for job to finish. Current status is {job_status}.")
    time.sleep(10)


In [ ]:
import requests

response = requests.get(transcript_uri)

pprint(response.json(), depth=2)


In [ ]:
print(response.json()["results"]["transcripts"][0]["transcript"])


In [ ]:
pprint(response.json()["results"]["items"])


## Translate

We can also translate languages.

![Translate Logo](images/translate.png)


In [ ]:
translate = aws_session.client(service_name="translate")

result = translate.translate_text(
    Text="Machine Learning is a very interesting subject",
    SourceLanguageCode="en",
    TargetLanguageCode="es"
)

print('Translated Text: ' + result.get('TranslatedText'))


## Comprehend

We can analyze text to find the meaning of it.

![Comprehend](images/comprehend.png)


In [ ]:
class ComprehendDetect:
    """Encapsulates Comprehend detection functions."""
    def __init__(self, comprehend_client):
        """
        :param comprehend_client: A Boto3 Comprehend client.
        """
        self.comprehend_client = comprehend_client


    def detect_languages(self, text):
        """
        Detects languages used in a document.

        :param text: The document to inspect.
        :return: The list of languages along with their confidence scores.
        """
        try:
            response = self.comprehend_client.detect_dominant_language(Text=text)
            languages = response['Languages']
        except ClientError:
            print("Couldn't detect languages.")
            raise
        else:
            return languages

    def detect_entities(self, text, language_code):
        """
        Detects entities in a document. Entities can be things like people and places
        or other common terms.

        :param text: The document to inspect.
        :param language_code: The language of the document.
        :return: The list of entities along with their confidence scores.
        """
        try:
            response = self.comprehend_client.detect_entities(
                Text=text, LanguageCode=language_code)
            entities = response['Entities']
        except ClientError:
            print("Couldn't detect entities.")
            raise
        else:
            return entities

    def detect_key_phrases(self, text, language_code):
        """
        Detects key phrases in a document. A key phrase is typically a noun and its
        modifiers.

        :param text: The document to inspect.
        :param language_code: The language of the document.
        :return: The list of key phrases along with their confidence scores.
        """
        try:
            response = self.comprehend_client.detect_key_phrases(
                Text=text, LanguageCode=language_code)
            phrases = response['KeyPhrases']
        except ClientError:
            print("Couldn't detect phrases.")
            raise
        else:
            return phrases

    def detect_pii(self, text, language_code):
        """
        Detects personally identifiable information (PII) in a document. PII can be
        things like names, account numbers, or addresses.

        :param text: The document to inspect.
        :param language_code: The language of the document.
        :return: The list of PII entities along with their confidence scores.
        """
        try:
            response = self.comprehend_client.detect_pii_entities(
                Text=text, LanguageCode=language_code)
            entities = response['Entities']
        except ClientError:
            print("Couldn't detect PII entities.")
            raise
        else:
            return entities

    def detect_sentiment(self, text, language_code):
        """
        Detects the overall sentiment expressed in a document. Sentiment can
        be positive, negative, neutral, or a mixture.

        :param text: The document to inspect.
        :param language_code: The language of the document.
        :return: The sentiments along with their confidence scores.
        """
        try:
            response = self.comprehend_client.detect_sentiment(
                Text=text, LanguageCode=language_code)
        except ClientError:
            print("Couldn't detect sentiment.")
            raise
        else:
            return response

    def detect_syntax(self, text, language_code):
        """
        Detects syntactical elements of a document. Syntax tokens are portions of
        text along with their use as parts of speech, such as nouns, verbs, and
        interjections.

        :param text: The document to inspect.
        :param language_code: The language of the document.
        :return: The list of syntax tokens along with their confidence scores.
        """
        try:
            response = self.comprehend_client.detect_syntax(
                Text=text, LanguageCode=language_code)
            tokens = response['SyntaxTokens']
        except ClientError:
            print("Couldn't detect syntax.")
            raise
        else:
            return tokens


In [ ]:
comprehend = ComprehendDetect(aws_session.client('comprehend'))

sample_text = "Machine learning is an important component of the growing field of data science. Through the use of statistical methods, algorithms are trained to make classifications or predictions, and to uncover key insights in data mining projects. These insights subsequently drive decision making within applications and businesses, ideally impacting key growth metrics. As big data continues to expand and grow, the market demand for data scientists will increase. They will be required to help identify the most relevant business questions and the data to answer them."

demo_size = 3

print("Detecting languages.")
languages = comprehend.detect_languages(sample_text)
pprint(languages)
lang_code = languages[0]['LanguageCode']

print("Detecting entities.")
entities = comprehend.detect_entities(sample_text, lang_code)
print(f"The first {demo_size} are:")
pprint(entities[:demo_size])

print("Detecting key phrases.")
phrases = comprehend.detect_key_phrases(sample_text, lang_code)
print(f"The first {demo_size} are:")
pprint(phrases[:demo_size])

print("Detecting personally identifiable information (PII).")
pii_entities = comprehend.detect_pii(sample_text, lang_code)
print(f"The first {demo_size} are:")
pprint(pii_entities[:demo_size])

print("Detecting sentiment.")
sentiment = comprehend.detect_sentiment(sample_text, lang_code)
print(f"Sentiment: {sentiment['Sentiment']}")
print("SentimentScore:")
pprint(sentiment['SentimentScore'])

print("Detecting syntax elements.")
syntax_tokens = comprehend.detect_syntax(sample_text, lang_code)
print(f"The first {demo_size} are:")
pprint(syntax_tokens[:demo_size])


## Sagemaker

![Sagemaker Logo](images/sagemaker.jpg)

There are also services that grant you access to a full Machine Learning work environment, including data storage, data processing, security, automated workflows, ephemeral compute resources, and services publishing.

![MLOps Platform](images/MLOps%20Platform.jpeg)

There is also a free service for learning Machine Learning and do simple experiments.

[Amazon Sagemaker Studio Lab](https://studiolab.sagemaker.aws)

## Hugging Face

If you use Sagemaker Studio Lab, or if you want to learn some great Machine Learning tools, read about [Hugging Face](https://huggingface.co).

![Hugging Face Logo](images/huggingface-logo.png)

## Other Machine Learning cloud providers

Of course AWS is not the only Cloud Provider. There are also very complete offers from Google Cloud Platform and Microsoft Azure.

And there are also "small" services that we can use, almost all of them with a free tier. Some of them are:

- [Dataiku](https://www.dataiku.com/product/)
- [Google Colab](https://colab.research.google.com)
- [JetBrains Datalore](https://datalore.jetbrains.com)
- [Kaggle](https://www.kaggle.com)
- [Paperspace](https://www.paperspace.com)

And of course, [Binder](https://mybinder.org), if only to see and run the notebooks.
